## Part 1: Creating a Scope Document from .dxf and Current BOM Files

### Scenario Overview:

We are a part of the *procurement automation team* at a manufacturing company working on a live sourcing requirement. The *design engineering team* has shared actual .dxf CAD drawings for a new industrial assembly, and we also have access to the *current BOM files* from the design management system.

Our first task is to *extract key components, specifications, and material details* from these files to generate a clean *Scope Document. This document becomes the backbone for the next stage: **RFQ (Request for Quotation) Preparation* and *intelligent vendor communication*.

### STEP 1: Scope Identification & Matching Engine

We first need to **understand what exactly we're sourcing**. This is done by parsing the **.dxf CAD drawings** and **BOM files** provided by the engineering team.

To do this, we built a **matching engine** that compares each item in the BOM against historical sourcing data using a **Weighted Similarity Strategy**.

### Matching Logic: Weighted Similarity Strategy

We compute item similarity using a weighted combination of the following fields:

| **Field**           | **Weight** | **Notes**                                                  |
| ------------------- | ---------- | ---------------------------------------------------------- |
| `Description`       | 0.4        | Fuzzy match of item name (e.g., "Control Valve", "Sensor") |
| `Specification`     | 0.3        | Exact or partial match (e.g., "316 Stainless Steel")       |
| `Material Category` | 0.2        | Must match exactly or be within related categories         |
| `Unit of Measure`   | 0.1        | Should match directly (e.g., "Each" vs "Feet")             |


--

Required packages

In [ ]:
import os
import sys

# adding the directory with the connection code to the system path
# so that we can import it
sys.path.append("../code")

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
# this cell isn't strictly necessary but it makes it easier to
# to update the code from llm_connect if you make changes
%reload_ext autoreload
%autoreload 2
%aimport functions

In [ ]:
import json
import requests
from langchain_openai import ChatOpenAI
from llm_connect import get_response
from dotenv import load_dotenv
import functions as fn

load_dotenv()


In [ ]:
client = ChatOpenAI(
    model="llama-3",
    api_key=os.getenv("LLAMA_API_KEY"),
    openai_api_base="https://traip13.tgptinf.ucsd.edu/v1",
)

!pip install rapidfuzz

In [ ]:
pip install oauth2client

## Multi-BOM Matching with DXF Linking - METHOD 1 (Using Difflib)

In [1]:
import os
import pandas as pd
from difflib import SequenceMatcher

# -----------------------------
# Load Historical BOMs
# -----------------------------
def load_historical_bom_data(folder_path="historical_data"):
    all_data = []
    for file in os.listdir(folder_path):
        if file.endswith(".xlsx"):
            df = pd.read_excel(os.path.join(folder_path, file))
            df["Source Historical BOM"] = os.path.splitext(file)[0]
            all_data.append(df)
    return pd.concat(all_data, ignore_index=True)


# -----------------------------
# Load a Single New BOM
# -----------------------------
def load_new_bom(file_path):
    return pd.read_excel(file_path)


# -----------------------------
# Weighted Similarity Function
# -----------------------------
def similarity(a, b):
    return SequenceMatcher(None, str(a).lower(), str(b).lower()).ratio()


def match_item(new_row, historical_df):
    best_match = None
    highest_score = 0

    for _, old_row in historical_df.iterrows():
        score = (
            0.4 * similarity(new_row["Description"], old_row["Description"]) +
            0.3 * similarity(new_row["Specification"], old_row["Specification"]) +
            0.2 * (1.0 if new_row["Material Category"] == old_row["Material Category"] else 0.0) +
            0.1 * (1.0 if new_row["Unit of Measure"] == old_row["Unit of Measure"] else 0.0)
        )
        if score > highest_score:
            highest_score = score
            best_match = old_row

    return {
        "Suggested Supplier": best_match["Supplier"] if best_match is not None else None,
        "Suggested Spec": best_match["Specification"] if best_match is not None else None,
        "Match Score": round(highest_score, 2),
        "Matched Item ID": best_match["Item ID"] if best_match is not None else None,
        "Source Historical BOM": best_match["Source Historical BOM"] if best_match is not None else None,
    }


# -----------------------------
# Generate Recommendations for All New BOM Files
# -----------------------------
def generate_recommendations_for_all_boms(new_bom_folder="generated_files", dxf_folder="generated_dxf"):
    historical_df = load_historical_bom_data()
    all_recommendations = []

    for bom_file in os.listdir(new_bom_folder):
        if bom_file.endswith(".xlsx"):
            new_bom_path = os.path.join(new_bom_folder, bom_file)
            new_bom_df = load_new_bom(new_bom_path)

            for _, new_row in new_bom_df.iterrows():
                result = match_item(new_row, historical_df)
                item_id = new_row["Item ID"]

                # Find matching DXF file
                dxf_file = next((f for f in os.listdir(dxf_folder)
                                 if f.endswith(".dxf") and item_id in f), None)
                dxf_path = os.path.join(dxf_folder, dxf_file) if dxf_file else None

                all_recommendations.append({
                    "Source New BOM": bom_file,
                    "Item ID": item_id,
                    "Description": new_row["Description"],
                    "Suggested Supplier": result["Suggested Supplier"],
                    "Suggested Spec": result["Suggested Spec"],
                    "Match Score": result["Match Score"],
                    "Matched Item ID": result["Matched Item ID"],
                    "Source Historical BOM": result["Source Historical BOM"],
                    "DXF File Path": dxf_path
                })

    return pd.DataFrame(all_recommendations)


# -----------------------------
# Run the Full Pipeline
# -----------------------------
if __name__ == "__main__":
    final_recommendations = generate_recommendations_for_all_boms(
        new_bom_folder="generated_files",     # Folder with BOM_1.xlsx ... BOM_5.xlsx
        dxf_folder="generated_dxf"            # Folder with DXF files
    )

    # Save or display
    final_recommendations.to_excel("BOM_Supplier_Recommendations.xlsx", index=False)
    print("✅ Recommendations saved to BOM_Supplier_Recommendations.xlsx")


✅ Recommendations saved to BOM_Supplier_Recommendations.xlsx


### ✅ Output Recommendation Includes:
- 🏷 **Suggested Supplier**
- 🛠 **Matched Specification**
- 📁 **Source BOM** (from historical data)
- 📐 **DXF File Path**  
  Example: `generated_dxf/BOM_1_ITM-1234.dxf`

## METHOD 2: RAPIDFUZZ

In [3]:
import os
import pandas as pd
from rapidfuzz import fuzz

# -----------------------------
# Load Historical BOMs
# -----------------------------
def load_historical_bom_data(folder_path="historical_data"):
    all_data = []
    for file in os.listdir(folder_path):
        if file.endswith(".xlsx"):
            df = pd.read_excel(os.path.join(folder_path, file))
            df["Source Historical BOM"] = os.path.splitext(file)[0]
            all_data.append(df)
    return pd.concat(all_data, ignore_index=True)

# -----------------------------
# Match Function Using RapidFuzz
# -----------------------------
def match_item_with_rapidfuzz(new_row, historical_df):
    """
    Matches a new BOM item against historical BOM dataset using token-based
    fuzzy string matching from `rapidfuzz`.

    Fields:
    - Description (0.4 weight): token sort fuzzy match
    - Specification (0.3): direct fuzzy match
    - Material Category (0.2): exact match
    - Unit of Measure (0.1): exact match
    """
    best_match = None
    highest_score = 0

    for _, old_row in historical_df.iterrows():
        score = (
            0.4 * (fuzz.token_sort_ratio(new_row["Description"], old_row["Description"]) / 100) +
            0.3 * (fuzz.ratio(new_row["Specification"], old_row["Specification"]) / 100) +
            0.2 * (1.0 if new_row["Material Category"] == old_row["Material Category"] else 0.0) +
            0.1 * (1.0 if new_row["Unit of Measure"] == old_row["Unit of Measure"] else 0.0)
        )

        if score > highest_score:
            highest_score = score
            best_match = old_row

    return {
        "Suggested Supplier": best_match["Supplier"] if best_match is not None else None,
        "Suggested Spec": best_match["Specification"] if best_match is not None else None,
        "Match Score": round(highest_score, 2),
        "Matched Item ID": best_match["Item ID"] if best_match is not None else None,
        "Source Historical BOM": best_match["Source Historical BOM"] if best_match is not None else None,
    }

# -----------------------------
# Run Recommendations for BOM_1 to BOM_5
# -----------------------------
def generate_recommendations_for_all_boms(new_bom_folder="generated_files", dxf_folder="generated_dxf"):
    historical_df = load_historical_bom_data()
    all_recommendations = []

    for bom_file in os.listdir(new_bom_folder):
        if bom_file.endswith(".xlsx") and bom_file.startswith("BOM_"):  # Only BOM_1 to BOM_5
            new_bom_path = os.path.join(new_bom_folder, bom_file)
            new_bom_df = pd.read_excel(new_bom_path)

            for _, new_row in new_bom_df.iterrows():
                result = match_item_with_rapidfuzz(new_row, historical_df)
                item_id = new_row["Item ID"]

                # Match corresponding DXF file
                dxf_file = next((f for f in os.listdir(dxf_folder)
                                 if f.endswith(".dxf") and item_id in f), None)
                dxf_path = os.path.join(dxf_folder, dxf_file) if dxf_file else None

                all_recommendations.append({
                    "Source New BOM": bom_file,
                    "Item ID": item_id,
                    "Description": new_row["Description"],
                    "Suggested Supplier": result["Suggested Supplier"],
                    "Suggested Spec": result["Suggested Spec"],
                    "Match Score": result["Match Score"],
                    "Matched Item ID": result["Matched Item ID"],
                    "Source Historical BOM": result["Source Historical BOM"],
                    "DXF File Path": dxf_path
                })

    return pd.DataFrame(all_recommendations)

# -----------------------------
# Main Execution
# -----------------------------
if __name__ == "__main__":
    final_df = generate_recommendations_for_all_boms(
        new_bom_folder="generated_files",
        dxf_folder="generated_dxf"
    )
    final_df.to_excel("BOM_Supplier_Recommendations_Rapidfuzz.xlsx", index=False)
    print("✅ Recommendations saved to All_BOM_Supplier_Recommendations.xlsx")


✅ Recommendations saved to All_BOM_Supplier_Recommendations.xlsx


## ⚔️ `rapidfuzz` vs `difflib` for BOM Matching -  Comparison Table

| Feature                     | `difflib.SequenceMatcher`                         | `rapidfuzz` (e.g., `token_sort_ratio`)               |
|-----------------------------|---------------------------------------------------|------------------------------------------------------|
| 🔁 **Word Order Awareness** | ❌ `"Valve Control"` ≠ `"Control Valve"`          | ✅ Handles reordered words intelligently              |
| ✏️ **Typo Tolerance**       | ❌ Minimal — not fuzzy in a real-world sense       | ✅ Robust to minor typos and formatting differences   |
| ⚡ **Speed**                | ❌ Slower, especially with many comparisons        | ✅ Much faster and scalable for larger datasets       |
| 📊 **Matching Quality**     | 🟡 Works on character-level diff                  | ✅ Token-based logic gives **more accurate scores**   |
| 📦 **Installation**         | ✅ Built-in (no install needed)                   | ❗Requires: `pip install rapidfuzz`                   |
| 🤖 **String Intelligence**  | ❌ Blind to synonyms, abbreviations, or structure | 🟡 Slightly better — but still not semantic           |

---

### Summary:
> **`rapidfuzz` is strongly preferred** for BOM item matching because:
> - It handles word order
> - It tolerates minor text differences
> - It is faster and more accurate
> - It better mimics how humans interpret similarity


### Why We Use Google Sheets as a Data Source

As part of our procurement automation initiative, we wanted the system to behave as close to a real-world production setup as possible — *without needing an actual production database connection during development*.

Once the design team shared the latest `.dxf` CAD files and BOM extracts, our goal was to start building a dynamic sourcing toolchain. But we hit a critical design question early on:

> **How do we simulate a live, editable vendor and item database — one that non-technical teams like procurement or design engineering could also update in real-time?**

Instead of hardcoding sample data or maintaining CSVs locally, we connected our tool directly to **Google Sheets** using the **Google Sheets API**.

#### The Reasoning Behind It:

* **Mimics a live database** — Sheets act like cloud-hosted tables.
* **Easily editable** — Procurement and engineering teams can add or edit vendors, materials, and specs without needing a developer.
* **Real-time updates** — Any change to the sheet is instantly reflected in our tool.
* **Accessible & collaborative** — Everyone in the team has access through a shared Google Drive.

By integrating with Google Sheets, we not only enabled dynamic data handling but also kept the tool flexible and aligned with how procurement teams often work in practice — *through spreadsheets shared on email, Slack, or Google Drive*.

This approach also makes our workflow easily extensible in the future: switching from Sheets to a real database (like PostgreSQL or Airtable) would be as simple as replacing the data-fetching function while keeping the rest of the logic intact.

In [15]:
def get_data(sheet_name: str) -> list:
    """
    Fetch data  from a Google Sheet.

    Parameters
    ----------
    sheet_name : str
        Name of the Google Sheet tab.

    Returns
    -------
    list of dict
        Each row in the sheet as a dictionary.
    """
    scope = [
    "https://spreadsheets.google.com/feeds", 
    "https://www.googleapis.com/auth/drive"
    ]
    creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
    gs_client = gspread.authorize(creds)  
    spreadsheet = gs_client.open_by_url("https://docs.google.com/spreadsheets/d/1ioqw78lMaf1tpFrU4QmcShZgl79bn6iqljnF3Uu8krU/edit?gid=244310017#gid=244310017/edit#gid=0")
    worksheet = spreadsheet.worksheet(sheet_name)
    data = worksheet.get_all_records()
    return data

In [16]:
def get_item_list_from_sheet(sheet_name: str = "items") -> list:
    raw_data = get_data(sheet_name)
    return [row["Item Name"] for row in raw_data if "Item Name" in row and row["Item Name"].strip()]
items = get_item_list_from_sheet("items")

In [17]:
def get_item_data_from_sheet(sheet_name: str = "items") -> list:
    """
    Fetches full item rows from the Google Sheet tab 'items'.

    Returns
    -------
    list of dicts:
        Each dict contains item metadata like:
        Item Name, Specification, Quantity, Unit of Measure, Delivery Date, Drawing Ref
    """
    return get_data(sheet_name)


In [18]:
item_data = get_item_data_from_sheet("items")


### STEP 2 Intelligent Vendor Suggestion Using LLM

Once the scope document is finalized and we know *which items* we need and *what vendors* we have on record, the next step is to match them intelligently — not manually.

Instead of using static rules or spreadsheets, we integrate a **large language model (LLM)** that reads through each vendor’s profile and recommends the best match for each item.

#### What Purpose Does This Serve?

* **Automates vendor shortlisting** using natural language reasoning
* **Evaluates nuanced criteria** like certifications, lead times, and expertise
* **Explains its choices** using short, interpretable bullet-point justifications
* **Handles repetition smartly** (groups items for the same vendor in a single row)

This makes the RFQ preparation phase dramatically faster and less error-prone — even for a large list of items or vendors.


In [29]:
def vendor_selector_template(user_input):
    return f"""
You are a procurement assistant tasked with selecting the most suitable vendor for each item listed.

You have access to vendor profiles that include:

Past history with our company

Relevant certifications

Area of expertise

Average lead time (in days)

Your task:
For each item, select the single best vendor based on the above criteria and the relevance of the vendor's profile to the item’s requirements.

Output Format:
Return the results in a markdown table with the following columns:

Item

Suggested Vendor

Reason for Selection (1–2 bullet points per row)

Additional Instructions:

If the same vendor is suitable for multiple items, list those item names together in the same row under the “Item” column (comma-separated).

Do not include any introductory or closing text.

Add a title above the table:
"Vendors suggestion based on Bechtel database"" 

Item to source: {user_input['item']}

Vendor list:
{user_input['vendor_data']}
"""

def suggest_vendors_llm(item_name: list, llm: str = "llama") -> str:
    vendor_data = get_data("vendors")

    prompt_input = {
        "item": item_name,
        "vendor_data": vendor_data
    }

    return get_response(
        input=prompt_input,
        template=vendor_selector_template,
        role="You are a procurement assistant helping select vendors for a specific item.",
        temperature=0.4,
        max_tokens=400,
        md=True,
        llm=llm
    )



#### Why Use Our Internal Database Instead of Web Scraping?

In high-value infrastructure projects like this one, **vendor credibility and historical performance** are critical. That’s why:

* Our **first preference is always to work with vendors we've done business with**.
* Our internal vendor data includes critical trust signals — *past work history, certifications, average lead times, and area of expertise* — that are often missing from public sources.
* This aligns with procurement best practices: prioritize reliability over novelty.

> In the future, we plan to extend this tool with internet-based scraping (e.g., LinkedIn, vendor portals), but **the current focus is on precision and control.**

In [30]:
suggest_vendors_llm(items)

# Vendors suggestion based on Bechtel database
| Item | Suggested Vendor | Reason for Selection |
| --- | --- | --- |
| Carbon Steel Flange, Gate Valve | Alpha Solutions 2 | * Expertise in Flanges and Valves, * Certifications include API Monogram and CE |
| HVAC Duct Connector | Trans Partners 13 | * Expertise in HVAC Duct Connectors, * Fast lead time of 12 days |
| Cable Tray, Signal Conduit, Control Cable | Metro Industries 1 | * Expertise in Control Cable, Signal Conduit, and Cable Tray, * Preferred vendor for 3 years |
| Earthing Strip, Fire Rated Door | Alpha Solutions 2 | * Expertise in Earthing Strip and Fire Rated Door, * Certifications include API Monogram and CE |
| Structural Steel Beam | Steel Solutions 16 | * Expertise in Structural Steel, * Preferred vendor for 3 years |
| Lighting Fixture - Tunnel Type | Metro Industries 9 | * Expertise in Lighting Fixture, * Fast lead time of 13 days |


###  Function Overview: `suggest_vendors_llm()`

```python
def suggest_vendors_llm(item_name: list, llm: str = "llama") -> str
```

* **Inputs**:

  * `item_name`: List of item names that need vendor recommendations
  * `llm`: (Optional) LLM engine to use; default is `"llama"`

* **Process**:

  * Loads vendor data from our internal Google Sheet
  * Builds a detailed prompt using `vendor_selector_template()`
  * Calls `get_response()` to query the LLM with this structured prompt

* **Returns**:

  * A markdown table with suggested vendors, item names, and 1–2 reasons per row
  * Once this is generated, it will be used to integrate into a our another tool, to generate RFQ email for each vendors

#### RFQ Formatter Tool

### **RFQ Email Generation Using LLM**

Once we've finalized the vendor-item assignments, the next critical step is to **draft Request for Quotation (RFQ) emails** — each tailored to a specific vendor, containing clear item specs, quantities, and delivery timelines.

Rather than writing these emails manually, we use a **Large Language Model (LLM)** to generate structured, professional RFQs that are instantly ready to send.

####  **What Purpose Does This Serve?**

*  **Automates RFQ drafting**
  Consistent, high-quality messaging — every time.

*  **Reduces turnaround time**
  Saves hours when sending to multiple vendors at once.

*  **Minimizes manual errors**
  Ensures no item is missed, no detail is inconsistent.

*  **Improves readability**
  Items are presented in a clean Markdown table for clarity.

In [1]:
def generate_rfq_document(project_info: dict, item_data: list, vendor_name: str = None) -> str:
    """
    Generate a vendor-ready RFQ document in plain text/markdown.

    Parameters
    ----------
    project_info : dict
        Contains project name, delivery address, submission deadline, and buyer contact info.

    item_data : list of dict
        List of items to be included in the RFQ. Each item should have fields:
        - Item Name, Specification, Quantity, Unit of Measure, Delivery Date, Drawing Ref

    vendor_name : str, optional
        If a specific vendor is being addressed, include their name in the greeting.

    Returns
    -------
    str
        A formatted RFQ text block ready to be emailed or saved.
    """


In [31]:
def rfq_formatter_prompt(inputs):
    vendor_name = inputs.get("vendor_name", "Vendor")
    project = inputs["project_info"]
    items = inputs["item_data"]

    item_lines = "\n".join(
        [f"- {item['Item Name']}: {item['Specification']} | Qty: {item['Quantity']} {item['Unit of Measure']} | Delivery by: {item['Delivery Date']} | Ref: {item['Drawing Ref']}"
         for item in items]
    )

    return f"""
You are a procurement assistant tasked with creating a vendor-ready RFQ.
Generate a professional, well-structured RFQ in markdown format with the following structure:

Greeting:
"Dear {vendor_name},"

Body:
- Brief project description using: {project['Project Name']}, located in {project['Location']}
- Include buyer contact info: {project['Engineering Lead']} (procurement@doha-metro.com)
- List the items requested as bullet points

Closing:
- State deadline for quotation submission: 1 week from today
- Ask the vendor to confirm receipt and reach out with questions

Item List:
{item_lines}

Only return the RFQ body. No commentary or explanation.
"""


### Gmail Integration (Work in progress)

Once the RFQ is generated, it’s **automatically inserted into a Gmail draft** using the Gmail API. This means:

* No more copy-pasting content manually
* Emails are pre-addressed and formatted
* The procurement team just clicks **Send**

This creates a seamless workflow from **scope definition → vendor matching → RFQ communication**, powered entirely by intelligent automation.

In [23]:
import json

# Load the JSON file
with open("/home/jovyan/git/rsm-mgta495-rsm-skaniappan-2/Sudhersan/project_context.json", "r") as file:
    project_info = json.load(file)

project_info

{'Project Name': 'Doha Metro Phase 2',
 'Project Code': 'QATAR-METRO-2025',
 'Location': 'Doha, Qatar',
 'Client': 'Qatar Rail',
 'Engineering Lead': 'Ahmed Al-Mansoori',
 'Budget Holder': 'Central Procurement Division',
 'Expected Completion': '2025-12-31',
 'Scope Area': ['Underground stations',
  'HVAC systems',
  'Piping systems',
  'Structural steel',
  'Cable trays',
  'Control rooms',
  'Lighting and electrical infrastructure'],
 'Working Language': 'English/Arabic',
 'Incoterms': 'CIF – Doha Port',
 'Payment Terms': 'Net 45',
 'Certifications Needed': ['ISO 9001',
  'ISO 14001',
  'QCS 2014',
  'GCC Standards'],
 'NDA Status': 'Signed'}

In [32]:
suggested_vendor_map = {
    "Alpha Solutions 2": ["Carbon Steel Flange", "Gate Valve", "Earthing Strip", "Fire Rated Door"],
    "Trans Partners 13": ["HVAC Duct Connector"],
    "Metro Industries 1": ["Cable Tray", "Signal Conduit", "Control Cable"],
    "Steel Solutions 16": ["Structural Steel Beam"],
    "Metro Industries 9": ["Lighting Fixture - Tunnel Type"]
}

item_data_all = get_item_data_from_sheet("items")

# Generate RFQ per vendor
for vendor, item_names in suggested_vendor_map.items():
    item_subset = [item for item in item_data_all if item["Item Name"] in item_names]

    rfq_text = get_response(
        input={
            "project_info": project_info,
            "item_data": item_subset,
            "vendor_name": vendor
        },
        template=rfq_formatter_prompt,
        role="You are a helpful assistant drafting procurement RFQs.",
        llm="llama",
        md=True
    )

    print(f"\n--- RFQ for {vendor} ---\n")
    print(rfq_text)

### Request for Quotation for Doha Metro Phase 2 Project
Dear Alpha Solutions 2,

We are reaching out from the procurement department of Doha Metro Phase 2, located in Doha, Qatar, regarding a request for quotation for various mechanical and electrical items. For any inquiries or clarifications, please do not hesitate to contact Ahmed Al-Mansoori at procurement@doha-metro.com.

The following items are requested:
* Carbon Steel Flange: ASTM A105, PN16, Raised Face | Qty: 150 EA | Delivery by: 2025-08-30 | Ref: DM-STN-MECH-001
* Gate Valve: API 600, 6-inch, WCB Body | Qty: 60 EA | Delivery by: 2025-08-30 | Ref: DM-STN-MECH-003
* Earthing Strip: Copper, 25mm x 3mm | Qty: 800 MTR | Delivery by: 2025-08-30 | Ref: DM-ELEC-018
* Fire Rated Door: 120 min fire resistance, single leaf, 900x2100mm | Qty: 35 EA | Delivery by: 2025-08-30 | Ref: DM-ARCH-021

Please submit your quotation within 1 week from the date of this request. We kindly request that you confirm receipt of this RFQ and reach out to us should you have any questions or require further clarification.


--- RFQ for Alpha Solutions 2 ---

None


### Request for Quotation for Doha Metro Phase 2 Project
Dear Trans Partners 13,

We are reaching out regarding the Doha Metro Phase 2 project, located in Doha, Qatar. For any inquiries or questions, please do not hesitate to contact Ahmed Al-Mansoori at procurement@doha-metro.com.

The following items are requested for the project:
* HVAC Duct Connector: Flexible, 300mm x 300mm, Class 1 | Qty: 200 EA | Delivery by: 2025-08-30 | Ref: DM-HVAC-004

Please submit your quotation within 1 week from the date of this request. We kindly request that you confirm receipt of this RFQ and reach out to us should you have any questions or require further clarification.


--- RFQ for Trans Partners 13 ---

None


### Request for Quotation for Doha Metro Phase 2 Project
Dear Metro Industries 1,

We are reaching out from the procurement department of Doha Metro, regarding the Doha Metro Phase 2 project, located in Doha, Qatar. For any inquiries or clarifications, please do not hesitate to contact Ahmed Al-Mansoori at procurement@doha-metro.com.

The following items are requested for the project:
* Cable Tray: Ladder Type, GI, 300mm x 50mm | Qty: 500 MTR | Delivery by: 2025-08-30 | Ref: DM-ELEC-015
* Control Cable: CU/PVC/PVC, 4C x 2.5 sq.mm | Qty: 1000 MTR | Delivery by: 2025-08-30 | Ref: DM-ELEC-022
* Signal Conduit: PVC, 50mm dia, ISI marked | Qty: 750 MTR | Delivery by: 2025-08-30 | Ref: DM-SIG-005

Please submit your quotation within 1 week from the date of this RFQ. We kindly request that you confirm receipt of this RFQ and reach out to us should you have any questions or require further clarification.


--- RFQ for Metro Industries 1 ---

None


### Request for Quotation for Doha Metro Phase 2 Project
Dear Steel Solutions 16,

We are reaching out from the procurement department of Doha Metro Phase 2, located in Doha, Qatar, regarding a potential supply opportunity. For any inquiries or clarifications, please do not hesitate to contact Ahmed Al-Mansoori at procurement@doha-metro.com.

The following items are requested for quotation:
* Structural Steel Beam: IPE 300, Hot Rolled, EN 10025 | Qty: 120 MTR | Delivery by: 2025-08-30 | Ref: DM-STRC-002

Please submit your quotation within 1 week from the date of this request. We kindly request that you confirm receipt of this RFQ and reach out to us should you have any questions or require further clarification.


--- RFQ for Steel Solutions 16 ---

None


### Request for Quotation for Doha Metro Phase 2 Project
Dear Metro Industries 9,

We are reaching out to request a quotation for the supply of materials for the Doha Metro Phase 2 project, located in Doha, Qatar. For any inquiries or clarification regarding this RFQ, please do not hesitate to contact Ahmed Al-Mansoori at procurement@doha-metro.com.

The following items are requested:
* Lighting Fixture - Tunnel Type: IP66, LED, 230V, 4000K | Qty: 220 EA | Delivery by: 2025-08-30 | Ref: DM-LGT-010

Please submit your quotation within 1 week from the date of this letter. We kindly request that you confirm receipt of this RFQ and reach out to us should you have any questions or require further clarification.


--- RFQ for Metro Industries 9 ---

None
